In [1]:
import torch

import utils

import warnings
warnings.filterwarnings('ignore')

In [2]:
# log file
model_id = 4
title = 'arl50'
log = utils.Logger(verbose=True, title=title)
log.logger.info("1-1 ARL50-{}".format(model_id))

[2022-04-06 15:42:01,018] 1-1 ARL50-4


## Data Preparation

In [3]:
# path of img data
pth_train = 'Data/ISIC2017/Aug_Training_Data'
pth_valid = 'Data/ISIC2017/ISIC-2017_Validation_Data'
pth_test = 'Data/ISIC2017/ISIC-2017_Test_Data'


ann_train = utils.Annotation('Data/ISIC2017/ISIC-2017_Training_Aug_Part3_GroundTruth.csv')
ann_valid = utils.Annotation('Data/ISIC2017/ISIC-2017_Validation_Part3_GroundTruth.csv')

In [4]:
from torchvision import transforms
from torch.utils import data

transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406],
                                                     [0.229, 0.224, 0.225])
                                ])

# augmentation transform
transform_train = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomVerticalFlip(p=0.5),
                                      transforms.RandomAffine(10, scale=(0.9, 1.1)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])
                                      ])

In [5]:
# create train dataset
train_data = utils.RandomPatch(ann_train.df, pth_train, transform=transform_train)
train_loader = data.DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True, num_workers=2)

# create validation dataset
valid_data = utils.RandomPatch(ann_valid.df, pth_valid, transform=transform)
valid_loader = data.DataLoader(valid_data, batch_size=32, shuffle=False, num_workers=2)

## Network Design

In [6]:
# GPU setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
import nets

model = nets.arlnet50(pretrained=True, num_classes=3)
# 1-2
# model = nets.resnet50(pretrained=True, num_classes=3)
# # 2
# model = nets.arlnet18(pretrained=False, num_calsses=3)

# model = utils.load_model(device, name="arl50.pkl")
# model = utils.load_state_dict(model, device, name="arl50_dict.pth")
model.to(device)

ARLNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Train
### Criterion

Focal Loss
$$
{\text{FL}(p_{t}) = - \alpha_t (1 - p_{t})^\gamma \log\left(p_{t}\right)} 
$$

In [8]:
# setting weights
# # 2-1/3
# class_weight = [1 for _ in range(ann_train.class_num)]
# 2-2/4
class_nums = ann_train.count_samples()
class_weight = [len(ann_train.df)/(len(class_nums)*num) for num in class_nums]

# 2-1/2
criterion = utils.FocalLoss(alpha=class_weight, gamma=2, num_classes=ann_train.class_num, reduction='sum')
# # 2-3/4
# criterion = nn.CrossEntropyLoss(weight=torch.Tensor(class_weight), reduction='sum')

log.logger.info(criterion)

[2022-04-06 15:42:03,920] Criterion: Focal Loss
 α = tensor([1., 1., 1.])
 γ = 2


### Train

The mini-batch SGD algorithm with a batch size of 32 was adopted as the optimizer. The learning rate was initialized to
0.01 for training ARL-CNN14 from scratch and 0.0001 for fine-tuning ARL-CNN50 with pre-trained parameters, and was
reduced by half very 30 epochs. The initial weighting factor of the attention feature maps was set to 0.001 in each ARL block
when fine-tuning the ARL-CNN50. The maximum epoch number was set to 100.

We used the officially provided validation set to monitor the performance of our model and stopped the training process when the network fell into overfitting.

In [9]:
from torch import optim

In [10]:
def train(model, train_loader, test_loader, max_epoch=100, test_period=5, early_threshold=5):
    """ train with a scheduler on learning rate

    Args:
        test_period (int): period of test
        early_threshold (int): threshold for early stoppig strategy, which pays attention to acc on test set
    """
    N_train = len(train_loader.dataset)
    patience = early_threshold
    eval_metrics = utils.Evaluation(device, ann_train.categories, 0)
    
    # make sure the model is in the training mode
    model.train()

    global epoch
    for epoch in range(epoch, max_epoch):
        cost = 0
        correct = 0
        
        for x, y in train_loader:
            # setting GPU
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            z = model(x)
            loss = criterion(z, y)
            loss.backward()
            optimizer.step()
            
            cost += loss.item()
            _, yhat = torch.max(z.data, 1)
            correct += (yhat == y).sum().item()
            
        cost = cost / N_train
        costs.append(cost)    # average cost
        

        # ! acc on train in train mode
        acc = correct / N_train
        train_accs.append(acc)

        # adjsut learning rate
        scheduler.step()


        if epoch % test_period == 0:
            eval_metrics.get_probs(model, test_loader)
            eval_metrics.complete_scores()
            test_accs.append(eval_metrics.acc)
            b_accs.append(eval_metrics.b_acc)
            f1_scores.append(eval_metrics.f1_score)
            auc = eval_metrics.roc_auc["micro"]
            auces.append(auc)
            
            if auc > eval_metrics.best_score:
                eval_metrics.best_score = auc
                patience = early_threshold
                # 在测试集上准确率上升时，保存模型参数
                utils.save_state_dict(model, name="{}_dict_{}.pth".format(title, model_id))
            else:
                patience -= 1
                if patience == 0:
                    break
                    
            # (每个epoch）保存checkpoint
            utils.check_train(log, model, optimizer, epoch, scheduler, pth_check='ch_train_{}.pth'.format(title))
            utils.check_eval(log, costs, train_accs, test_accs, b_accs, f1_scores, auces, pth_check='ch_eval_{}.pth'.format(title), verbose=False)
                
            log.logger.info("{:3d} cost: {:.4f}\ttrain_acc: {:.4f}\ttest_acc: {:.4f}\ttest_bacc: {:.4f}\tf1_score: {}\tauc: {:.4f}".format(
                epoch, cost, acc, test_accs[-1], b_accs[-1], f1_scores[-1], auces[-1]))

            # change back to training mode    
            model.train()

In [11]:
# setting
## filename
model_file = '{}-{}.pkl'.format(title, model_id)

## hyper-params
init_lr = 1e-4
weight_decay = 1e-3
max_epoch = 100
test_period = 1
early_threshold = 20

## optimizer: https://pytorch.org/docs/stable/optim.html
momentum = 0.9
optimizer = optim.SGD(model.parameters(), lr=init_lr, momentum=momentum, weight_decay=weight_decay)


## learning rate decay
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

# 初始化训练状态
epoch = utils.load_train(log, model, optimizer, scheduler)
costs, train_accs, test_accs, b_accs, f1_scores, auces = utils.load_eval(log)

In [ ]:
# # load from checkpoint 从checkpoint加载继续训练
# epoch = utils.load_train(log, model, optimizer, scheduler, pth_check='ch_train_arl.pth')
# costs, train_accs, test_accs, b_accs, f1_scores, auces = utils.load_eval(log, pth_check='ch_eval_arl.pth')

log.logger.info("Training...\n{}\n{}".format(optimizer, scheduler))
train(model, train_loader, valid_loader, max_epoch, test_period, early_threshold)

[2022-04-06 15:42:03,951] Training...
SGD (
Parameter Group 0
    dampening: 0
    initial_lr: 0.0001
    lr: 0.0001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.001
)
[2022-04-06 15:47:13,855] Saving training checkpoint at checkpoint/ch_train_arl50.pth


State dict has been saved to ./model/temp/arl50_dict_4.pth


[2022-04-06 15:47:14,085]   0 cost: 0.3141	train_acc: 0.6437	test_acc: 0.6133	test_bacc: 0.4618	f1_score: [0.11764705882352941, 0.5, 0.7281553398058251]	auc: 0.8217
[2022-04-06 15:52:18,696] Saving training checkpoint at checkpoint/ch_train_arl50.pth


State dict has been saved to ./model/temp/arl50_dict_4.pth


[2022-04-06 15:52:18,971]   1 cost: 0.2586	train_acc: 0.6943	test_acc: 0.6467	test_bacc: 0.5200	f1_score: [0.45000000000000007, 0.43636363636363634, 0.7414634146341464]	auc: 0.8543
[2022-04-06 15:57:27,500] Saving training checkpoint at checkpoint/ch_train_arl50.pth


...

[2022-04-06 15:57:27,855]   2 cost: 0.2852	train_acc: 0.6669	test_acc: 0.6667	test_bacc: 0.5526	f1_score: [0.29268292682926833, 0.647887323943662, 0.7553191489361704]	auc: 0.8657
[2022-04-06 16:02:33,966] Saving training checkpoint at checkpoint/ch_train_arl50.pth
[2022-04-06 16:02:34,242]   3 cost: 0.2210	train_acc: 0.7211	test_acc: 0.6533	test_bacc: 0.6471	f1_score: [0.46511627906976744, 0.7462686567164178, 0.7210884353741496]	auc: 0.8290
[2022-04-06 16:07:41,525] Saving training checkpoint at checkpoint/ch_train_arl50.pth


State dict has been saved to ./model/temp/arl50_dict_4.pth


[2022-04-06 16:07:41,801]   4 cost: 0.2233	train_acc: 0.7075	test_acc: 0.7933	test_bacc: 0.7088	f1_score: [0.4878048780487804, 0.8470588235294119, 0.8390804597701149]	auc: 0.9191
[2022-04-06 16:12:45,515] Saving training checkpoint at checkpoint/ch_train_arl50.pth
[2022-04-06 16:12:45,798]   5 cost: 0.2047	train_acc: 0.7346	test_acc: 0.7333	test_bacc: 0.6779	f1_score: [0.5573770491803278, 0.7042253521126761, 0.8095238095238095]	auc: 0.8700
[2022-04-06 16:17:59,883] Saving training checkpoint at checkpoint/ch_train_arl50.pth


State dict has been saved to ./model/temp/arl50_dict_4.pth


[2022-04-06 16:18:00,174]   6 cost: 0.2012	train_acc: 0.7467	test_acc: 0.7867	test_bacc: 0.7182	f1_score: [0.5333333333333333, 0.8470588235294119, 0.8235294117647058]	auc: 0.9228


In [ ]:
# # save checkpoint
# utils.check_train(log, model, optimizer, epoch, scheduler)
# utils.check_eval(log, costs, train_accs, test_accs, b_accs, f1_scores, auces)

In [ ]:
# save model
utils.save_model(model, name=model_file)
log.logger.info("Filename: {}\ncosts = {}\ntrain_accs = {}\ntest_acc = {}\nb_accs = {}\nf1_scores = {}\nauces = {}".format(
    model_file, costs, train_accs, test_accs, b_accs, f1_scores, auces))

## Evaluation

Implemented in [Evaluation.ipynb](Evaluation.ipynb)

In the test stage, we used the same patch extraction method to randomly crop nine patches from each test image, fed them to the trained network, and averaged the obtained scores as the predicted score of the image.

In [ ]:
# create test dataset
ann_test = utils.Annotation('Data/ISIC2017/ISIC-2017_Test_v2_Part3_GroundTruth.csv')
test_data = utils.RandomPatch(ann_test.df, pth_test, transform=transform)
test_loader = data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
eval_metrics = utils.Evaluation(device, ann_test.categories)


def evaluation_report(model, data_loader):
    y, prob = eval_metrics.get_probs(model, data_loader)
    probs = [prob]
    for _ in range(8):
        _, prob = eval_metrics.get_probs(model, data_loader)
        probs.append(prob)
    prob = np.mean(probs, axis=0)
    eval_metrics.prob = prob
    
    eval_metrics.complete_scores()
    
    log.logger.info(eval_metrics.report)
    roc_auc = eval_metrics.roc_auc
    
    for key in roc_auc.keys():
        log.logger.info("AUC({}): {}".format(key, roc_auc[key]))

In [ ]:
evaluation_report(model, test_loader)